# 03 - Federated Learning

## Defines

Define the available types of federated learning.

In [88]:
### THIS SECTION NEEDS TO BE SET TO DETERMINE WHICH CONFIGURATION METHOD TO UTILISE

SPLIT_AVAILABLE_METHODS = ['INDIVIDUAL_ATTACK', 'ATTACK_GROUP', 'STRATIFIED']
METHOD = 'STRATIFIED'
NUM_OF_STRATIFIED_CLIENTS = 10 # only applies to stratified method

Include the defines for the dataframe columns and the attack labels and their mappings

In [89]:
from includes import X_columns, y_column, dict_34_classes, dict_8_classes, dict_2_classes

##  Imports

In [90]:
%%capture
%pip install flwr[simulation] torch torchvision matplotlib sklearn openml

In [91]:
import os
import pandas as pd
import numpy as np
import flwr as fl
from tqdm import tqdm
import warnings
#warnings.filterwarnings('ignore')

import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from flwr.common import Metrics
from torch.utils.data import DataLoader, random_split


In [92]:
print("flwr", fl.__version__)
print("numpy", np.__version__)
print("torch", torch.__version__)

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Training on {DEVICE}")

flwr 1.4.0
numpy 1.24.2
torch 1.13.1
Training on cuda:0


## Load the Dataset

In [93]:
DATASET_DIRECTORY = '../datasets/CICIoT2023/'

## Training data

Either read the training pickle file if it exists, or process the dataset from scratch.

In [94]:
# Check to see if the file 'training_data.pkl' exists in the directory. If it does, load it. If not, print an error.
if os.path.isfile('training_data.pkl'):
    print("File exists, loading data...")
    train_df = pd.read_pickle('training_data.pkl')
    print("Training data loaded from pickle file.")

else:
    df_sets = [k for k in os.listdir(DATASET_DIRECTORY) if k.endswith('.csv')]
    df_sets.sort()
    training_sets = df_sets[:int(len(df_sets)*.8)]
    test_sets = df_sets[int(len(df_sets)*.8):]

    # Print the number of files in each set
    print('Training sets: {}'.format(len(training_sets)))
    print('Test sets: {}'.format(len(test_sets)))

    ######################
    # TEMP CODE
    ######################
    # Set training_sets to the last entry of training_sets
    training_sets = training_sets[-1:]
    print(f"HACK TO REPLICATE ORIGINAL AUTHORS CODE WITH ONE FILE TRAIN - {training_sets}")
    ######################
    # END TEMP CODE
    ######################

    # Concatenate all training sets into one dataframe
    dfs = []
    print("Reading training data...")
    for train_set in tqdm(training_sets):
        df_new = pd.read_csv(DATASET_DIRECTORY + train_set)
        dfs.append(df_new)
    train_df = pd.concat(dfs, ignore_index=True)

    # Map y column to the dict_34_classes values - The pickle file already has this done.
    train_df['label'] = train_df['label'].map(dict_34_classes)

    # Save the output to a pickle file
    print("Writing training data to pickle file...")
    train_df.to_pickle('training_data.pkl')

print("Training data size: {}".format(train_df.shape))


File exists, loading data...
Training data loaded from pickle file.
Training data size: (243649, 47)


In [95]:
train_df

,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
0,0.000838,54.62,6.05,64.00,11.961779,11.961779,0.0,0.0,0.0,0.0,...,0.111473,54.45,8.307598e+07,9.5,10.392912,0.037895,0.035900,0.02,141.55,5
1,0.005486,75.88,6.00,64.00,29.502125,29.502125,0.0,0.0,1.0,0.0,...,0.100314,54.24,8.309325e+07,9.5,10.395361,0.143036,0.346802,0.03,141.55,3
2,0.000000,0.00,45.61,65.81,151.517376,151.517376,0.0,0.0,0.0,0.0,...,57.165223,576.80,8.369379e+07,9.5,33.783684,80.958879,8638.780727,0.40,141.55,17
3,0.000000,54.00,6.00,64.00,1.500542,1.500542,0.0,0.0,1.0,0.0,...,0.000000,54.00,8.309408e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,3
4,0.004568,745.42,5.95,65.13,8.082100,8.082100,0.0,0.0,0.0,0.0,...,549.190629,927.04,8.333561e+07,9.5,41.550978,776.661367,318084.344439,0.95,141.55,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243644,0.000000,54.00,6.00,64.00,19.582485,19.582485,0.0,0.0,0.0,0.0,...,0.000000,54.00,8.331443e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,2
243645,0.037146,78.22,36.21,63.18,24.542045,24.542045,0.0,0.0,0.0,0.0,...,110.233513,453.78,8.358187e+07,9.5,30.338676,154.660856,23401.960226,0.53,141.55,18
243646,3.293075,1025996.92,17.00,64.00,572.160392,572.160392,0.0,0.0,0.0,0.0,...,0.000000,554.00,8.378910e+07,9.5,33.286634,0.000000,0.000000,0.00,141.55,19
243647,0.047343,35223.00,17.00,64.00,15083.107398,15083.107398,0.0,0.0,0.0,0.0,...,0.000000,50.00,8.309852e+07,9.5,10.000000,0.000000,0.000000,0.00,141.55,4


### Scale the training data input features

In [96]:
scaler = StandardScaler()
train_df[X_columns] = scaler.fit_transform(train_df[X_columns])

---
## Test Data
Concat the test data into a single dataframe

In [97]:
# Check to see if the file 'test_data.pkl' exists in the directory. If it does, load it. If not, print an error.
if os.path.isfile('testing_data.pkl'):
    print("File exists, loading data...")
    test_df = pd.read_pickle('testing_data.pkl')
    print("Test data loaded from pickle file.")

else:
    df_sets = [k for k in os.listdir(DATASET_DIRECTORY) if k.endswith('.csv')]
    df_sets.sort()
    training_sets = df_sets[:int(len(df_sets)*.8)]
    test_sets = df_sets[int(len(df_sets)*.8):]

    # Print the number of files in each set
    print('Test sets: {}'.format(len(test_sets)))
    
    # Concatenate all testing sets into one dataframe
    dfs = []
    print("Reading test data...")
    for test_set in tqdm(test_sets):
        df_new = pd.read_csv(DATASET_DIRECTORY + test_set)
        dfs.append(df_new)
    test_df = pd.concat(dfs, ignore_index=True)

    # Map y column to the dict_34_classes values - The pickle file already has this done.
    test_df['label'] = test_df['label'].map(dict_34_classes)

    # Save the output to a pickle file
    print("Writing test data to pickle file...")
    test_df.to_pickle('testing_data.pkl')

print("Testing data size: {}".format(test_df.shape))

Test sets: 34
Reading test data...


100%|██████████| 34/34 [00:27<00:00,  1.22it/s]


Writing test data to pickle file...
Testing data size: (10340161, 47)


### Scale the testing data input features

In [98]:
scaler = StandardScaler()
test_df[X_columns] = scaler.fit_transform(test_df[X_columns])

---


In [99]:
print("Training data size: {}".format(train_df.shape))
print("Testing data size: {}".format(test_df.shape))

Training data size: (243649, 47)
Testing data size: (10340161, 47)


Stratify the input dataset